In [169]:
import pandas as pd

school_data_path = "../Resources/schools_complete.csv"
student_data_path = "../Resources/students_complete.csv"

school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [170]:
district_school_count = len(school_data_complete["school_name"].unique())
district_student_count = school_data_complete["Student ID"].count()
district_budget = school_data_complete["budget"].unique().sum()
district_math_score = school_data_complete["math_score"].mean()
district_reading_score = school_data_complete["reading_score"].mean()

masked_passed_math = school_data_complete["math_score"] >= 70
masked_passed_reading = school_data_complete["reading_score"] >= 70
masked_passed_overall = (school_data_complete["math_score"] >= 70) & \
                        (school_data_complete["reading_score"] >= 70)

district_math_count = school_data_complete[masked_passed_math].count()["student_name"]
district_reading_count = school_data_complete[masked_passed_reading].count()["student_name"]
district_overall_count = school_data_complete[masked_passed_overall].count()["student_name"]

district_math_percent = district_math_count / district_student_count * 100
district_reading_percent = district_reading_count / district_student_count * 100
district_overall_percent = district_overall_count / district_student_count * 100

district_overall_percent

65.17232575950983

In [171]:
district_summary = pd.DataFrame([{
    "Total Schools" : district_school_count,
    "Total Students" : district_student_count,
    "Total Budget" : district_budget,
    "Average Math Score" : district_math_score,
    "Average Reading Score" : district_reading_score,
    "Percent Passing Math" : district_math_percent,
    "Percent Passing Reading" : district_reading_percent,
    "Percent Overall Passing" : district_overall_percent
    }])

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}".format)
district_summary["Percent Passing Math"] = district_summary["Percent Passing Math"].map("{:.2f}%".format)
district_summary["Percent Passing Reading"] = district_summary["Percent Passing Reading"].map("{:.2f}%".format)
district_summary["Percent Overall Passing"] = district_summary["Percent Overall Passing"].map("{:.2f}%".format)

district_summary.style.hide_index()

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


In [186]:
#look back on this from starter code
school_types = pd.DataFrame(school_data.set_index(["school_name"])["type"])
school_types

,type
school_name,
Huang High School,District
Figueroa High School,District
Shelton High School,Charter
Hernandez High School,District
Griffin High School,Charter
Wilson High School,Charter
Cabrera High School,Charter
Bailey High School,District
Holden High School,Charter


In [173]:
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

In [184]:
school_data_byschool = school_data_complete.groupby(["school_name"])
school_average = school_data_byschool.mean()

per_school_counts = school_average[["size"]]
per_school_budget = school_average[["budget"]]
per_school_capita = pd.DataFrame(per_school_budget["budget"] / per_school_counts["size"])
per_school_math = school_average[["math_score"]]
per_school_reading = school_average[["reading_score"]]

per_school_passed_math_count = school_data_complete[masked_passed_math].groupby(["school_name"]).count()
per_school_passed_reading_count = school_data_complete[masked_passed_reading].groupby(["school_name"]).count()
per_school_passed_overall_count = school_data_complete[masked_passed_overall].groupby(["school_name"]).count()

per_school_passed_math = pd.DataFrame((per_school_passed_math_count["math_score"] / per_school_counts["size"]) * 100)
per_school_passed_reading = pd.DataFrame((per_school_passed_reading_count["reading_score"] / per_school_counts["size"]) * 100)
per_school_passed_overall = pd.DataFrame((per_school_passed_overall_count["Student ID"] / per_school_counts["size"]) * 100)

In [187]:
per_school_summary = pd.merge(school_types, per_school_counts, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_budget, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_capita, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_math, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_reading, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_passed_math, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_passed_reading, on = "school_name")
per_school_summary = pd.merge(per_school_summary, per_school_passed_overall, on = "school_name")

per_school_summary

# per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
# per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

,type,size,budget,0_x,math_score,reading_score,0_y,0_x,0_y
school_name,,,,,,,,,
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761.0,1056600.0,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
